# Day 5
## Part 1

In [1]:
#r "nuget: FSharp.UMX"

open FSharp.UMX

type [<Measure>] seed
type [<Measure>] soil
type [<Measure>] fertilizer
type [<Measure>] water
type [<Measure>] light
type [<Measure>] temperature
type [<Measure>] humidity
type [<Measure>] location

type MapRange<[<Measure>]'source, [<Measure>]'destination> = 
    { DestinationStart : int<'destination>; SourceStart : int<'source>; RangeSize : int }

type InputSection =
    | SeedList of list<int<seed>>
    | SeedToSoilMap of MapRange<seed, soil>[]
    | SoilToFertilizerMap of MapRange<soil, fertilizer>[]
    | FertilizerToWaterMap of MapRange<fertilizer, water>[]
    | WaterToLightMap of MapRange<water, light>[]
    | LightToTemperatureMap of MapRange<light, temperature>[]
    | TemperatureToHumidityMap of MapRange<temperature, humidity>[]
    | HumidityToLocationMap of MapRange<humidity, location>[]



Installed Packages FSharp.UMX, 1.1.0

In [11]:
#r "nuget:FParsec"

open FParsec

let space = pchar ' '
let newline = pchar '\n'
let int_list = sepBy pint32 space
let seedList = 
    (pstring "seeds: " >>. int_list .>> newline)
    |>> List.map UMX.tag<seed>
    |>> SeedList

let inline map_line<[<Measure>]'source, [<Measure>]'destination> = 
    pint32 .>> space .>>. pint32 .>> space .>>. pint32
    |>> fun ((a,b),c) -> { DestinationStart = UMX.tag<'destination> a; SourceStart = UMX.tag<'source> b; RangeSize = c }

let inline sortMap (ranges : MapRange<'source, 'destination> list) = 
    ranges
    |> List.sortBy (fun r -> r.SourceStart)
    |> Array.ofList

let seedToSoil = 
    (pstring "seed-to-soil map:" >>. newline >>. sepEndBy1 map_line<seed,soil> newline)
    |>> (sortMap >> SeedToSoilMap)

let soilToFertilizer = 
    (pstring "soil-to-fertilizer map:" >>. newline >>. sepEndBy1 map_line<soil, fertilizer> newline)
    |>> (sortMap >> SoilToFertilizerMap)

let fertilizerToWater = 
    (pstring "fertilizer-to-water map:" >>. newline >>. sepEndBy1 map_line<fertilizer, water> newline)
    |>> (sortMap >> FertilizerToWaterMap)

let waterToLight =
    (pstring "water-to-light map:" >>. newline >>. sepEndBy1 map_line<water, light> newline)
    |>> (sortMap >> WaterToLightMap)

let lightToTemperature =
    (pstring "light-to-temperature map:" >>. newline >>. sepEndBy1 map_line<light, temperature> newline)
    |>> (sortMap >> LightToTemperatureMap)

let temperatureToHumidity =
    (pstring "temperature-to-humidity map:" >>. newline >>. sepEndBy1 map_line<temperature, humidity> newline)
    |>> (sortMap >> TemperatureToHumidityMap)

let humidityToLocation =
    (pstring "humidity-to-location map:" >>. newline >>. sepEndBy1 map_line<humidity, location> newline)
    |>> (sortMap >> HumidityToLocationMap)

let section =
    seedList <|>
    seedToSoil <|>
    soilToFertilizer <|>
    fertilizerToWater <|>
    waterToLight <|>
    lightToTemperature <|>
    temperatureToHumidity <|>
    humidityToLocation

let sections : Parser<InputSection list, unit> = sepBy section newline


Installed Packages FParsec, 1.1.1

In [ ]:
open System
open System.Collections

type BinarySearchIndex = | Match of int | BeforeFirst | AfterLast of int | Between of int * int

let binarySearchBy<'T,'U when 'U :> IComparable<'U>> (sel : 'T -> 'U) (x:'U) (values:'T[]) =
    let compare = System.Collections.Generic.GenericComparer.Default.Compare
    let comparer = { new IComparer with member __.Compare(a, b) = compare (a :?> 'T |> sel, x) }
    let i = Array.BinarySearch(values, x, comparer)
    if i >= 0 then
        Match i
    else
        let iNext = ~~~i
        if iNext = 0 then
            BeforeFirst
        elif iNext = values.Length then
            AfterLast (iNext-1)
        else
            Between (iNext-1, iNext)

let applyMap (ranges: MapRange<'src, 'dst>[]) (src: int<'src>) : int<'dst> =
    let index = binarySearchBy (fun r -> r.SourceStart) src ranges
    match index with
    | BeforeFirst -> UMX.cast<'src,'dst>src
    | AfterLast i
    | Match i
    | Between (i, _) -> 
        let range = ranges[i]
        let offset = UMX.untag<'src>(src - range.SourceStart)
        if offset < range.RangeSize then
            range.DestinationStart + UMX.tag<'dst>offset
        else
            UMX.cast<'src,'dst>src


In [12]:
type Model = 
    {
        Seeds : list<int<seed>>
        SeedToSoilMap : MapRange<seed, soil>[]
        SoilToFertilizerMap : MapRange<soil, fertilizer>[]
        FertilizerToWaterMap : MapRange<fertilizer, water>[]
        WaterToLightMap : MapRange<water, light>[]
        LightToTemperatureMap : MapRange<light, temperature>[]
        TemperatureToHumidityMap : MapRange<temperature, humidity>[]
        HumidityToLocationMap : MapRange<humidity, location>[]
    }

module Model =
    let empty = 
        {
            Seeds = []
            SeedToSoilMap = [||]
            SoilToFertilizerMap = [||]
            FertilizerToWaterMap = [||]
            WaterToLightMap = [||]
            LightToTemperatureMap = [||]
            TemperatureToHumidityMap = [||]
            HumidityToLocationMap = [||]
        }
    
    let applySection section model =
        match section with
        | SeedList seeds -> { model with Seeds = seeds }
        | SeedToSoilMap map -> { model with SeedToSoilMap = map }
        | SoilToFertilizerMap map -> { model with SoilToFertilizerMap = map }
        | FertilizerToWaterMap map -> { model with FertilizerToWaterMap = map }
        | WaterToLightMap map -> { model with WaterToLightMap = map }
        | LightToTemperatureMap map -> { model with LightToTemperatureMap = map }
        | TemperatureToHumidityMap map -> { model with TemperatureToHumidityMap = map }
        | HumidityToLocationMap map -> { model with HumidityToLocationMap = map }

    let fromSections sections =
        sections |> List.fold (fun model section -> applySection section model) empty

    let parse =
        sections
        |>> fromSections

    let getLocation model (seed: int<seed>) =
        seed
        |> applyMap model.SeedToSoilMap
        |> applyMap model.SoilToFertilizerMap
        |> applyMap model.FertilizerToWaterMap
        |> applyMap model.WaterToLightMap
        |> applyMap model.LightToTemperatureMap
        |> applyMap model.TemperatureToHumidityMap
        |> applyMap model.HumidityToLocationMap


In [17]:
#r "nuget: FsUnit.Xunit"

open FsUnitTyped

match run seedList "seeds: 79 14 55 13\n" with
| Failure (msg,_,_) -> failwith msg
| Success (section,_,_) ->
    section |> shouldEqual (SeedList [79<seed>; 14<seed>; 55<seed>; 13<seed>])

match run Model.parse "seed-to-soil map:\n50 98 2\n52 50 48\n" with
| Failure (msg,_,_) -> failwith msg
| Success (model,_,_) ->
    [79<seed>; 14<seed>; 55<seed>; 13<seed>]
    |> List.map (applyMap model.SeedToSoilMap)
    |> shouldEqual [81<soil>; 14<soil>; 57<soil>; 13<soil>]

let testInput = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

match run Model.parse testInput with
| Failure (msg,_,_) -> failwith msg
| Success (model,_,_) ->
    model.Seeds |> shouldEqual [79<seed>; 14<seed>; 55<seed>; 13<seed>]
    
    model.Seeds
    |> List.map (Model.getLocation model)
    |> shouldEqual [82<location>; 43<location>; 86<location>; 35<location>]


Installed Packages FsUnit.Xunit, 5.6.1

Error: Xunit.Sdk.EqualException: Assert.Equal() Failure: Values differ
Expected: Equals [81; 43; 86; 35]
Actual:   [82; 43; 86; 35]
   at FsUnit.Xunit.raiseEqualException@16[b](StringDescription description, String value) in D:\github\repos\FsUnit\src\FsUnit.Xunit\FsUnit.fs:line 17
   at FsUnit.Xunit.Assert.That.Static[a](a actual, IMatcher`1 matcher) in D:\github\repos\FsUnit\src\FsUnit.Xunit\FsUnit.fs:line 27
   at FsUnitTyped.TopLevelOperators.shouldEqual[a](a expected, a actual) in D:\github\repos\FsUnit\src\FsUnit.Xunit\FsUnitTyped.fs:line 14
   at <StartupCode$FSI_0040>.$FSI_0040.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeWithNoArgs(Object obj, BindingFlags invokeAttr)